<a href="https://colab.research.google.com/github/sriharikrishna/EuroAD26/blob/main/EuroAD_deepxde_odetest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[DeepXDE](https://deepxde.readthedocs.io/en/latest/) is a library for scientific machine learning and physics-informed learning.

We start by installing it.

In [ ]:
!pip install deepxde

First the imports

In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
import deepxde as dde
import numpy as np

This is an ODE Solver system provided as an [example](https://deepxde.readthedocs.io/en/latest/demos/pinn_forward/ode.system.html).

In [ ]:
def ode_system(x, y):
    """ODE system.
    dy1/dx = y2
    dy2/dx = -y1
    """
    # Most backends
    y1, y2 = y[:, 0:1], y[:, 1:]
    dy1_x = dde.grad.jacobian(y, x, i=0)
    dy2_x = dde.grad.jacobian(y, x, i=1)
    # Backend jax
    # y_val, y_fn = y
    # y1, y2 = y_val[:, 0:1], y_val[:, 1:]
    # dy1_x, _ = dde.grad.jacobian(y, x, i=0)
    # dy2_x, _ = dde.grad.jacobian(y, x, i=1)
    return [dy1_x - y2, dy2_x + y1]


def boundary(_, on_initial):
    return on_initial


def func(x):
    """
    y1 = sin(x)
    y2 = cos(x)
    """
    return np.hstack((np.sin(x), np.cos(x)))

Run the system using `float` data types

In [ ]:
val = 1.0
val1 = 0.0
val2 = 1.0

geom = dde.geometry.TimeDomain(0, val)
ic1 = dde.icbc.IC(geom, lambda x: val1, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: val2, boundary, component=1)
data = dde.data.PDE(geom, ode_system, [ic1, ic2], 35, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [2]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=20000)

#dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Running the same code with complex data does not work

In [ ]:
val = 1.0 + 2j
val1 = 0.0 + 2j
val2 = 1.0 + 2j

geom = dde.geometry.TimeDomain(0, val)
ic1 = dde.icbc.IC(geom, lambda x: val1, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: val2, boundary, component=1)
data = dde.data.PDE(geom, ode_system, [ic1, ic2], 35, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [2]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=20000)

So we have to look elsewhere